In [13]:
import numpy as np
from qiskit.algorithms.linear_solvers.numpy_linear_solver import NumPyLinearSolver
from qiskit.algorithms.linear_solvers.hhl import HHL
matrix = np.array([[1, -1/3], [-1/3, 1]])
vector = np.array([1, 0])
naive_hhl_solution = HHL().solve(matrix, vector)

In [14]:
classical_solution = NumPyLinearSolver().solve(matrix, vector / np.linalg.norm(vector))

In [15]:
from qiskit.algorithms.linear_solvers.matrices.tridiagonal_toeplitz import TridiagonalToeplitz
tridi_matrix = TridiagonalToeplitz(1, 1, -1 / 3)

tridi_solution = HHL().solve(tridi_matrix, vector)

In [16]:
print('classical state:', classical_solution.state)

classical state: [1.125 0.375]


In [17]:
print('naive state:')
print(naive_hhl_solution.state)
print('tridiagonal state:')
print(tridi_solution.state)

naive state:
        ┌──────────────┐┌──────┐        ┌─────────┐
  q178: ┤ circuit-1102 ├┤3     ├────────┤3        ├
        └──────────────┘│      │┌──────┐│         │
q179_0: ────────────────┤0     ├┤2     ├┤0        ├
                        │  QPE ││      ││  QPE_dg │
q179_1: ────────────────┤1     ├┤1     ├┤1        ├
                        │      ││  1/x ││         │
q179_2: ────────────────┤2     ├┤0     ├┤2        ├
                        └──────┘│      │└─────────┘
  q180: ────────────────────────┤3     ├───────────
                                └──────┘           
tridiagonal state:
        ┌──────────────┐┌──────┐        ┌─────────┐
  q200: ┤ circuit-1315 ├┤3     ├────────┤3        ├
        └──────────────┘│      │┌──────┐│         │
q201_0: ────────────────┤0     ├┤2     ├┤0        ├
                        │  QPE ││      ││  QPE_dg │
q201_1: ────────────────┤1     ├┤1     ├┤1        ├
                        │      ││  1/x ││         │
q201_2: ────────────────┤2     ├

In [18]:
print('classical Euclidean norm:', classical_solution.euclidean_norm)
print('naive Euclidean norm:', naive_hhl_solution.euclidean_norm)
print('tridiagonal Euclidean norm:', tridi_solution.euclidean_norm)

classical Euclidean norm: 1.1858541225631423
naive Euclidean norm: 1.1858541225631407
tridiagonal Euclidean norm: 1.185854122563139


In [19]:
from qiskit.quantum_info import Statevector

naive_sv = Statevector(naive_hhl_solution.state).data
tridi_sv = Statevector(tridi_solution.state).data

# Extract the right vector components. 1000 corresponds to the index 8 and 1001 corresponds to the index 9
naive_full_vector = np.array([naive_sv[8], naive_sv[9]])
tridi_full_vector = np.array([tridi_sv[8], tridi_sv[9]])

print('naive raw solution vector:', naive_full_vector)
print('tridi raw solution vector:', tridi_full_vector)

naive raw solution vector: [-8.32667268e-17-2.05817839e-16j  2.08166817e-17+4.00437992e-16j]
tridi raw solution vector: [-2.84494650e-16+8.28864894e-17j -8.32667268e-17+6.02722261e-16j]


In [20]:
naive_full_vector = np.real(naive_full_vector)
tridi_full_vector = np.real(tridi_full_vector)

In [21]:
print('full naive solution vector:', naive_hhl_solution.euclidean_norm*naive_full_vector/np.linalg.norm(naive_full_vector))
print('full tridi solution vector:', tridi_solution.euclidean_norm*tridi_full_vector/np.linalg.norm(tridi_full_vector))
print('classical state:', classical_solution.state)

full naive solution vector: [-1.15044748  0.28761187]
full tridi solution vector: [-1.13810856 -0.33310494]
classical state: [1.125 0.375]


In [22]:
from scipy.sparse import diags

num_qubits = 2
matrix_size = 2 ** num_qubits
# entries of the tridiagonal Toeplitz symmetric matrix
a = 1
b = -1/3

matrix = diags([b, a, b], [-1, 0, 1], shape=(matrix_size, matrix_size)).toarray()
vector = np.array([1] + [0]*(matrix_size - 1))

# run the algorithms
classical_solution = NumPyLinearSolver().solve(matrix, vector / np.linalg.norm(vector))
naive_hhl_solution = HHL().solve(matrix, vector)
tridi_matrix = TridiagonalToeplitz(num_qubits, a, b)
tridi_solution = HHL().solve(tridi_matrix, vector)

print('classical euclidean norm:', classical_solution.euclidean_norm)
print('naive euclidean norm:', naive_hhl_solution.euclidean_norm)
print('tridiagonal euclidean norm:', tridi_solution.euclidean_norm)

classical euclidean norm: 1.237833351044751
naive euclidean norm: 1.2099806231119121
tridiagonal euclidean norm: 1.209457721870593


In [23]:
from qiskit import transpile

num_qubits = list(range(1,5))
a = 1
b = -1/3

i=1
# calculate the circuit depths for different number of qubits to compare the use of resources
naive_depths = []
tridi_depths = []
for nb in num_qubits:
    matrix = diags([b, a, b], [-1, 0, 1], shape=(2**nb, 2**nb)).toarray()
    vector = np.array([1] + [0]*(2**nb -1))
    
    naive_hhl_solution = HHL().solve(matrix, vector)
    tridi_matrix = TridiagonalToeplitz(nb, a, b)
    tridi_solution = HHL().solve(tridi_matrix, vector)

    naive_qc = transpile(naive_hhl_solution.state,basis_gates=['id', 'rz', 'sx', 'x', 'cx'])
    tridi_qc = transpile(tridi_solution.state,basis_gates=['id', 'rz', 'sx', 'x', 'cx'])
    
    naive_depths.append(naive_qc.depth())
    tridi_depths.append(tridi_qc.depth())
    i +=1

In [24]:
sizes = [str(2**nb)+"x"+str(2**nb) for nb in num_qubits]
columns = ['size of the system', 'quantum_solution depth', 'tridi_solution depth']
data = np.array([sizes, naive_depths, tridi_depths])
row_format ="{:>23}" * (len(columns) + 2)
for team, row in zip(columns, data):
    print(row_format.format(team, *row))

     size of the system                    2x2                    4x4                    8x8                  16x16
 quantum_solution depth                    334                   2562                  34071                 404844
   tridi_solution depth                    565                   5107                  14756                  46552


In [25]:
print('excess:', [naive_depths[i] - tridi_depths[i] for i in range(0, len(naive_depths))])

excess: [-231, -2545, 19315, 358292]
